In [2]:
from astropy.table import *
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from astropy.table import hstack
import os
import seaborn as sns
from scipy.stats import binned_statistic
import matplotlib
import matplotlib.backends.backend_pdf
from scipy.stats import chi2, binned_statistic
from cosmic_calc import * 
from scipy.optimize import curve_fit
from bisect import bisect
from datetime import datetime
from scipy.optimize import curve_fit
from statsmodels.stats.weightstats import DescrStatsW

plt.rc('font', family='serif'), plt.rc('xtick', labelsize=18), plt.rc('ytick', labelsize=18)
plt.rcParams['savefig.dpi'] = 300
plt.rc('text',usetex=True)
matplotlib.rcParams['xtick.direction'] = 'in'
matplotlib.rcParams['ytick.direction'] = 'in'
np.set_printoptions(precision=3)

In [3]:
dev_path = '/Users/lejay/research/lephare_dev/my_code/'
output_dir = dev_path+'output_cats/'
graham_output_dir = dev_path + 'graham_output_cats_new/'

#### Trim some columns

In [68]:
prefix = 'pcat_' # '','v9pcat'

# cat_names = ['COSMOS_deep','ELAIS_deep']
cat_names = ['COSMOS_deep','DEEP_deep','ELAIS_deep','XMM-LSS_deep']
# cat_names = ['XMM-LSS_deep']
for cat_name in cat_names:
    print(graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_nonir_il_nz_prior_formass.fits')
    if cat_name != 'XMM-LSS_deep':
        output_cat = Table.read(graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_nonir_il_nz_prior_formass.fits')
    else:
        output_cat = Table.read(graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_noirac_il_nz_prior_formass.fits')
    
    keys_set1 = [key for key in output_cat.keys() if 'NBAND_USED' in key]
    keys_set2 = [key for key in output_cat.keys() if 'CONTEXT' in key]
    keys_set3 = [key for key in output_cat.keys() if 'Z_BEST' in key]
    keys_set4 = [key for key in output_cat.keys() if 'Z_ML' in key]
    keys_other = ['IDENT','CONTEXT','NBAND_USED','EBV_BEST', 'STRING_INPUT']
    if cat_name == 'COSMOS_deep':
        keys_other.append('HSC_i_MAGERR_AUTO')
    else:
        keys_other.append('FLAG_FIELD')
    keys = keys_set1 + keys_set2 + keys_set3 + keys_set4+ keys_other
    
    for key in keys:
        try:
            output_cat.remove_column(key)
        except KeyError:
            print(key, 'already removed')
        
    print(len(output_cat.keys()))
    print('')
    
    # rename some columns
    for key in output_cat.keys():
        if '_massrun' in key:
            output_cat.rename_column(key,key.replace('_massrun',''))
    
    if cat_name != 'XMM-LSS_deep':
        output_cat.write(graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_nonir_il_nz_prior_formass.fits', overwrite=True)
    else:
        output_cat.write(graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_noirac_il_nz_prior_formass.fits', overwrite=True)


/Users/lejay/research/lephare_dev/my_code/graham_output_cats_new/pcat_COSMOS_deep_cat_out_nomaglimit_nonir_il_nz_prior_formass.fits
IDENT already removed
CONTEXT already removed
NBAND_USED already removed
EBV_BEST already removed
STRING_INPUT already removed
HSC_i_MAGERR_AUTO already removed
103

/Users/lejay/research/lephare_dev/my_code/graham_output_cats_new/pcat_DEEP_deep_cat_out_nomaglimit_nonir_il_nz_prior_formass.fits
IDENT already removed
CONTEXT already removed
NBAND_USED already removed
EBV_BEST already removed
STRING_INPUT already removed
FLAG_FIELD already removed
84

/Users/lejay/research/lephare_dev/my_code/graham_output_cats_new/pcat_ELAIS_deep_cat_out_nomaglimit_nonir_il_nz_prior_formass.fits
IDENT already removed
CONTEXT already removed
NBAND_USED already removed
EBV_BEST already removed
STRING_INPUT already removed
FLAG_FIELD already removed
84

/Users/lejay/research/lephare_dev/my_code/graham_output_cats_new/pcat_XMM-LSS_deep_cat_out_nomaglimit_nonir_il_nz_prior_forma

#### Add photometric flags

In [69]:
# the function to add photometric flags
# for a certain band, good flag requirements are detected + low uncertainty

def add_photo_flag(cat,cat_name):
    cat_matched = cat
    
    max_phot_err = 0.2
    cond_U = ( (cat_matched['u'] > 0.) & (cat_matched['u'] < 50.) & (cat_matched['u_err'] > 0.) & (cat_matched['u_err'] < max_phot_err*2) )| \
             ( (cat_matched['uS'] > 0.) & (cat_matched['uS'] < 50.) & (cat_matched['uS_err'] > 0.) & (cat_matched['uS_err'] < max_phot_err*2) )
    cond_g = ( (cat_matched['g'] > 0.) & (cat_matched['g'] < 50.) & (cat_matched['g_err'] > 0.) & (cat_matched['g_err'] < max_phot_err*2) ) 
    cond_r = ( (cat_matched['r'] > 0.) & (cat_matched['r'] < 50.) & (cat_matched['r_err'] > 0.) & (cat_matched['r_err'] < max_phot_err*2) )
    cond_i = ( (cat_matched['i'] > 0.) & (cat_matched['i'] < 50.) & (cat_matched['i_err'] > 0.) & (cat_matched['i_err'] < max_phot_err) )
    cond_z = ( (cat_matched['z'] > 0.) & (cat_matched['z'] < 50.) & (cat_matched['z_err'] > 0.) & (cat_matched['z_err'] < max_phot_err) )
    cond_y = ( (cat_matched['y'] > 0.) & (cat_matched['y'] < 50.) & (cat_matched['y_err'] > 0.) & (cat_matched['y_err'] < max_phot_err) )
    flag_optical = cond_U.astype(int) + cond_g.astype(int) + cond_r.astype(int) + cond_i.astype(int) + cond_z.astype(int) + cond_y.astype(int)
    
    if cat_name == 'COSMOS_deep' or cat_name == 'XMM-LSS_deep':
        cond_Y = ( (cat_matched['Yv'] > 0.) & (cat_matched['Yv'] < 50.) & (cat_matched['Yv_err'] > 0.) & (cat_matched['Yv_err'] < max_phot_err) )
        cond_J = ( (cat_matched['J'] > 0.) & (cat_matched['J'] < 50.) & (cat_matched['J_err'] > 0.) & (cat_matched['J_err'] < max_phot_err) )
        cond_H = ( (cat_matched['H'] > 0.) & (cat_matched['H'] < 50.) & (cat_matched['H_err'] > 0.) & (cat_matched['H_err'] < max_phot_err) )
        cond_Ks = ( (cat_matched['Ks'] > 0.) & (cat_matched['Ks'] < 50.) & (cat_matched['Ks_err'] > 0.) & (cat_matched['Ks_err'] < max_phot_err) )
        flag_nir = cond_Y.astype(int) + cond_J.astype(int) + cond_H.astype(int) + cond_Ks.astype(int)
    else:
        flag_nir = np.zeros(len(cat))
    
    if cat_name != 'XMM-LSS_deep':
        cond_ch1 = ( (cat_matched['MAG_AUTO_CH1_CORR'] > 0.) & (cat_matched['MAG_AUTO_CH1_CORR'] < 50.) & (cat_matched['MAGERR_AUTO_CH1_CORR'] < 4) )
        cond_ch2 = ( (cat_matched['MAG_AUTO_CH2_CORR'] > 0.) & (cat_matched['MAG_AUTO_CH2_CORR'] < 50.) & (cat_matched['MAGERR_AUTO_CH1_CORR'] < 4) )
        flag_irac = cond_ch1.astype(int) + cond_ch2.astype(int) 
    else:
        flag_irac = np.zeros(len(cat))
     
    flag_list = [flag_optical,flag_nir,flag_irac]
    flag_name_list = ['flag_optical','flag_nir','flag_irac']
        
    if len(cat_matched) != len(flag_optical) or len(cat_matched) != len(flag_nir) or len(cat_matched) != len(flag_irac):
        print('lengths do not match')
        return None
    else:
        col_flag_optical = Column(name='flag_optical', data=flag_optical)
        col_flag_nir = Column(name='flag_nir', data=flag_nir)
        col_flag_irac = Column(name='flag_irac', data=flag_irac)
        if 'flag_optical' in cat_matched.keys():
            cat_matched.remove_column('flag_optical')
        if 'flag_nir' in cat_matched.keys():
            cat_matched.remove_column('flag_nir')
        if 'flag_irac' in cat_matched.keys():
            cat_matched.remove_column('flag_irac')
            
        cat_matched.add_columns([col_flag_optical, col_flag_nir, col_flag_irac])
        return cat_matched

In [70]:
prefix = 'pcat_'
for cat_name in cat_names:
    print(cat_name)
    
    # read catalog
    if cat_name != 'XMM-LSS_deep':
        output_cat = Table.read(graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_nonir_il_nz_prior_formass.fits')
    else:
        output_cat = Table.read(graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_noirac_il_nz_prior_formass.fits')
    
    # add photometry flags
    output_cat = add_photo_flag(output_cat,cat_name)
    
    # write catalog
    if cat_name != 'XMM-LSS_deep':
        output_cat.write(graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_nonir_il_nz_prior_formass.fits', overwrite=True)
    else:
        output_cat.write(graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_noirac_il_nz_prior_formass.fits', overwrite=True)

COSMOS_deep
DEEP_deep
ELAIS_deep
XMM-LSS_deep


#### Add inside IRAC flag

In [71]:
cat_names = ['COSMOS_deep','DEEP_deep','ELAIS_deep']
filenames = ['IRAC_cosmos.reg','IRAC_deep2.reg','IRAC_elais.reg']

prefix = 'pcat_'
for i,cat_name in enumerate(cat_names):
    print(cat_name)
    region_file = '/Users/lejay/research/regions/'+filenames[i]
    cat_name_full = graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_nonir_il_nz_prior_formass.fits'
    cat = Table.read(cat_name_full)
    
    if 'inside_irac' in cat.keys():
        cat.remove_column('inside_irac')
        cat.write(cat_name_full, overwrite=True)
        
    os.system('/Users/lejay/Downloads/venice-master/bin/venice -m '+region_file+' -f all -cat '+cat_name_full+' -xcol RA -ycol DEC -flagName inside_irac -o !'+cat_name_full)
    
    cat = Table.read(cat_name_full)
    cat['inside_irac'] = (1-cat['inside_irac']).astype(bool)
    cat.write(cat_name_full, overwrite=True)


COSMOS_deep
DEEP_deep
ELAIS_deep


#### Add inside_uS_deep for XMM-LSS
Deep u* region

In [80]:
region_file = '/Users/lejay/research/regions/'+'MegaCam_uS_deep_xmm.reg'
cat_name_full = graham_output_dir+'pcat_XMM-LSS_deep_cat_out_nomaglimit_noirac_il_nz_prior_formass.fits'
cat = Table.read(cat_name_full)

if 'inside_uS_deep' in cat.keys():
    cat.remove_column('inside_uS_deep')
    cat.write(cat_name_full, overwrite=True)
    
os.system('/Users/lejay/Downloads/venice-master/bin/venice -m '+region_file+' -f all -cat '+cat_name_full+' -xcol RA -ycol DEC -flagName inside_uS_deep -o !'+cat_name_full)

cat = Table.read(cat_name_full)
cat['inside_uS_deep'] = (1-cat['inside_uS_deep']).astype(bool)
cat.write(cat_name_full, overwrite=True)


#### Add column for outshing correction

In [73]:
cat_names = ['COSMOS_deep','DEEP_deep','ELAIS_deep','XMM-LSS_deep']

prefix = 'pcat_'
m_keyname = 'MASS_MED'
for i,cat_name in enumerate(cat_names):
    if cat_name != 'XMM-LSS_deep':
        cat_name_full = graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_nonir_il_nz_prior_formass.fits'
    else:
        cat_name_full = graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_noirac_il_nz_prior_formass.fits'
    cat = Table.read(cat_name_full)
    
    # make outshing corrected mass (Sorba+Sawicki18)
    ssfr_list = cat['SSFR_MED']
    m_corr = (ssfr_list<=-9.23)*(0.02*ssfr_list+0.264)+(ssfr_list>-9.23)*(0.21*ssfr_list**2+3.9344*ssfr_list+18.505)
    mass_corrected = cat[m_keyname]+m_corr
    new_col_name = 'MASS_OUTSHINE'
    m_corrrected_col = Column(data=mass_corrected,name=new_col_name)
    
    # add new column
    if new_col_name in cat.keys():
        cat.remove_column(new_col_name)
    cat.add_column(m_corrrected_col)
    cat.write(cat_name_full, overwrite=True)


#### add more photometry flags

In [74]:
cat_names = ['COSMOS_deep','DEEP_deep','ELAIS_deep','XMM-LSS_deep']
# cat_names = ['XMM-LSS_deep']

prefix = 'pcat_' # or 'pcat_z6B_'
for cat_name in cat_names:
    if cat_name != 'XMM-LSS_deep':
        cat_name_full = graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_nonir_il_nz_prior_formass.fits'
        cat_name_phot = '/Users/lejay/phosphorus_catalogs/p_cats_published/'+cat_name+'_pcat_mags_gals_CH1_CH2_extinction_corrected.fits'
    else:
        cat_name_full = graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_noirac_il_nz_prior_formass.fits'
        cat_name_phot = '/Users/lejay/phosphorus_catalogs/p_cats_published/'+cat_name+'_pcat_mags_gals_CH1_CH2_extinction_corrected.fits'

    cat = Table.read(cat_name_full)
    combined_cat_name = cat_name_full

    if cat_name != 'XMM-LSS_deep':
        col_list = "'ID i_psf i_cmodel cmodel_fail_flag snr_Ch1 snr_CH2'"
        col_list_arr = ['i_psf','i_cmodel','cmodel_fail_flag','snr_CH1','snr_CH2']
    else:
        col_list = "'ID i_psf i_cmodel cmodel_fail_flag'"
        col_list_arr = ['i_psf','i_cmodel','cmodel_fail_flag']

    # remove the columns if they are alrady added 
    for col in col_list_arr:
        if col in cat.keys():
            cat.remove_column(col)
            
    for key in cat.keys():
        if 'inside_irac' in key or 'inside_uS_deep' in key:
            cat.remove_column(key)
            
    cat.write(cat_name_full, overwrite=True)
    
    # match and add columns
    print(cat_name)
    cmd = 'java -jar /Users/lejay/research/massive_gals/stilts.jar tmatch2 matcher=exact \
        in1='+cat_name_full + ' values1="ID" \
        in2='+cat_name_phot+' values2="ID"  join=1and2 icmd2="keepcols '+col_list+'" \
        out='+ combined_cat_name
    check = os.system(cmd)
    print(check)

    if check == 0:
        cat = Table.read(cat_name_full)
        cat.remove_column('ID_2')
        cat.rename_column('ID_1','ID')
        cat.write(cat_name_full, overwrite=True)
    else:
        print('Not matched!')

COSMOS_deep
0
DEEP_deep
0
ELAIS_deep
0
XMM-LSS_deep
0


#### add sfq info and flags

In [1]:
# color-color boundary
def boundary_basic(color1, ic_1, ic_2, slope):
    nuv_1 = ic_1
    nuv_2 = slope*color1 + ic_2
    return np.maximum(nuv_1, nuv_2)


def boundary_4d(color1, ic_1, ic_2, slope, z_factor, M_factor, z, M):
    nuv_1 = ic_1
    nuv_2 = (slope+z_factor*z+M_factor*M)*(color1 + ic_2)
    return np.maximum(nuv_1, nuv_2)


def boundary_3d(color1, ic_1, ic_2, slope, M_factor, M):
    nuv_1 = ic_1
    nuv_2 = (slope+M_factor*M)*color1 + ic_2
    return np.maximum(nuv_1, nuv_2)

In [18]:
# sfq_info in catalog
# keynames: sfq_nuvrk_myrun_[...]

prefix = 'pcat_'
sfq_fit_type = 'free'
add_sfq = 'add_sfq' # sfq_only or add_sfq
cat_type = 'all' # all or central_cosmos or xmm

##########################################
if prefix == 'pcat_' or prefix == 'pcat_z6B_':
    z_keyname = 'Z_COMBINE'
else:
    z_keyname = 'Z_ML'

detect_limit = 'nolimit'
photoz_type_show = ''
if cat_type == 'all':
    cat_names = ['COSMOS_deep','DEEP_deep','ELAIS_deep']
    # cat_names = ['COSMOS_deep','ELAIS_deep']
    phot_configs = ['nonir']
elif cat_type == 'xmm':
    cat_names = ['XMM-LSS_deep']
    phot_configs = ['noirac']
elif cat_type == 'central_cosmos' or 'c20_matched':
    cat_names = ['COSMOS_deep']
    phot_configs = ['nonir']
# phot_configs = ['nonirirac','nonir','noirac','allphot']

In [19]:
###########################################
for cat_name in cat_names:
    for phot_config in phot_configs:
        if cat_type == 'all': # all or central_cosmos
            if detect_limit == 'nolimit':
                cat_full_name = graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_'+phot_config+'_il_nz_prior_formass.fits'  # all fields   
            else:
                cat_full_name = graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_'+phot_config+'_il_nz_prior_'+detect_limit+'_formass.fits'  # all fields   
        elif cat_type == 'xmm':
            cat_full_name = graham_output_dir+prefix+cat_name+'_cat_out_nomaglimit_'+phot_config+'_il_nz_prior_formass.fits'
        elif cat_type == 'central_cosmos' and 'pcat' not in prefix:
            cat_full_name = graham_output_dir+'cat_out_08squdeg_'+phot_config+'_il_nz_prior_'+detect_limit+'_formass.fits' # central cosmos
        elif cat_type == 'central_cosmos' and 'pcat' in prefix:
            cat_full_name = graham_output_dir+'pcat_cat_out_08squdeg_'+phot_config+'_il_nz_prior_'+detect_limit+'_formass.fits' # pcat central cosmos
        elif cat_type == 'c20_matched':
            cat_full_name = cat_mycosmos_name.replace('_sfq_added.fits','_c20added.fits')
        else:
            cat_full_name = '/Volumes/LejaySSD/pcat_lephare_masses/COSMOS_galaxies_240408_c20added.fits'
            
        cat = Table.read(cat_full_name)
        # cat = cat[:100]
        sfq_types = ['fc_weighted','fm_weighted','balanced','onebin_fc','onebin_fm','onebin_bal']
        if add_sfq == 'sfq_only':
            cat = cat['ID','RA','DEC','Z_COMBINE','MASS_MED','i','MAG_ABS_r','MAG_ABS_Ks','MAG_ABS_NUV']
        
        print(cat_full_name)
        
        #nuvrk
        diagram_type = 'NUVrK'
        print(cat_name, phot_config, sfq_fit_type)
        
        for sfq_type in sfq_types:
            print(sfq_type)
            sfq_nuvrk = []
            for i in tqdm(range(len(cat))):
                gal = cat[i]
                
                if abs(gal['MAG_ABS_r']) < 50. and abs(gal['MAG_ABS_Ks'])< 50. and abs(gal['MAG_ABS_NUV'])< 50.:
                    nuvr = gal['MAG_ABS_NUV'] - gal['MAG_ABS_r']
                    rk = gal['MAG_ABS_r'] - gal['MAG_ABS_Ks']
                    
                    if 'onebin' not in sfq_type:
                        r_factors_base = np.array([1.1,1.1,1.0,0.9])
                        if sfq_type == 'fc_weighted':
                            r_factors = r_factors_base*2.5
                        elif sfq_type == 'fm_weighted':
                            r_factors = r_factors_base/2
                        elif sfq_type == 'balanced':
                            r_factors = r_factors_base
                        else:
                            raise NameError('wrong sfq type')
                            
                        if gal[z_keyname]>0.2 and gal[z_keyname]<=0.5:
                            r = r_factors[0]
                            best_boxes_nuvrk = np.load('best_params_4d_ssfr'+photoz_type_show+'_fcfm_'+str(r)+'_0.2_0.5.npy')
                            delta_gv_nuvrk = nuvr - boundary_4d(rk,best_boxes_nuvrk[0],best_boxes_nuvrk[1],best_boxes_nuvrk[2],best_boxes_nuvrk[3],best_boxes_nuvrk[4],gal[z_keyname],gal['MASS_MED'])
                            if delta_gv_nuvrk<0:
                                sfq_nuvrk.append(1.) # sf
                            else:
                                sfq_nuvrk.append(0.) # q
                                
                        elif gal[z_keyname]>0.5 and gal[z_keyname]<=0.8:
                            r = r_factors[1]
                            best_boxes_nuvrk = np.load('best_params_4d_ssfr'+photoz_type_show+'_fcfm_'+str(r)+'_0.5_0.8.npy')
                            delta_gv_nuvrk = nuvr - boundary_4d(rk,best_boxes_nuvrk[0],best_boxes_nuvrk[1],best_boxes_nuvrk[2],best_boxes_nuvrk[3],best_boxes_nuvrk[4],gal[z_keyname],gal['MASS_MED'])
                            if delta_gv_nuvrk<0:
                                sfq_nuvrk.append(1.) # sf
                            else:
                                sfq_nuvrk.append(0.) # q
                                
                        elif gal[z_keyname]>0.8 and gal[z_keyname]<=1.1:
                            r = r_factors[2]
                            best_boxes_nuvrk = np.load('best_params_4d_ssfr'+photoz_type_show+'_fcfm_'+str(r)+'_0.8_1.1.npy')
                            delta_gv_nuvrk = nuvr - boundary_4d(rk,best_boxes_nuvrk[0],best_boxes_nuvrk[1],best_boxes_nuvrk[2],best_boxes_nuvrk[3],best_boxes_nuvrk[4],gal[z_keyname],gal['MASS_MED'])
                            if delta_gv_nuvrk<0:
                                sfq_nuvrk.append(1.) # sf
                            else:
                                sfq_nuvrk.append(0.) # q
                        elif gal[z_keyname]>1.1 and gal[z_keyname]<=1.5:
                            r = r_factors[3]
                            best_boxes_nuvrk = np.load('best_params_4d_ssfr'+photoz_type_show+'_fcfm_'+str(r)+'_1.1_1.5.npy')
                            delta_gv_nuvrk = nuvr - boundary_4d(rk,best_boxes_nuvrk[0],best_boxes_nuvrk[1],best_boxes_nuvrk[2],best_boxes_nuvrk[3],best_boxes_nuvrk[4],gal[z_keyname],gal['MASS_MED'])
                            if delta_gv_nuvrk<0:
                                sfq_nuvrk.append(1.) # sf
                            else:
                                sfq_nuvrk.append(0.) # q
                        else:
                            sfq_nuvrk.append(-99.0) 
                    else:
                        if sfq_type == 'onebin_fc':
                            r = 2.5
                        elif sfq_type == 'onebin_fm':
                            r = 1.0
                        elif sfq_type == 'onebin_bal':
                            r = 0.5
                        else:
                            r = 1.0
                            
                        if gal[z_keyname]>0.1 and gal[z_keyname]<=1.5:
                            best_boxes_nuvrk = np.load('best_params_4d_ssfr'+photoz_type_show+'_fcfm_'+str(r)+'_z15.npy') # overall sfq boundary from 0.1-1.1 references
                            delta_gv_nuvrk = nuvr - boundary_4d(rk,best_boxes_nuvrk[0],best_boxes_nuvrk[1],best_boxes_nuvrk[2],best_boxes_nuvrk[3],best_boxes_nuvrk[4],gal[z_keyname],gal['MASS_MED'])
                            if delta_gv_nuvrk<0:
                                sfq_nuvrk.append(1.) # sf
                            else:
                                sfq_nuvrk.append(0.) # q
                        else:
                            sfq_nuvrk.append(99.)
    
                else:
                    sfq_nuvrk.append(99.)
                
            sfq_col_name = 'sfq_nuvrk_'+sfq_type
            sfq_col_nuvrk = Column(name=sfq_col_name, data=sfq_nuvrk)  # 1=sf, 0=q
            print(len(sfq_col_nuvrk))
            if sfq_col_name in cat.keys():
                cat.remove_column(sfq_col_name)
            cat.add_column(sfq_col_nuvrk)
            
            # high mass flag (fake massive flag)
            if 'XMM' not in cat_name:
                cond_ch1 = ( (cat['MAG_AUTO_CH1_CORR'] > 0.) & (cat['MAG_AUTO_CH1_CORR'] < 50.) & (cat['MAGERR_AUTO_CH1_CORR'] < 4.0))
                cond_ch2 = ( (cat['MAG_AUTO_CH2_CORR'] > 0.) & (cat['MAG_AUTO_CH2_CORR'] < 50.) & (cat['MAGERR_AUTO_CH1_CORR'] < 4.0))
                flag_irac = cond_ch1.astype(int) + cond_ch2.astype(int)
                
                fake_massive_cuts = np.array([9.71,9.92,10.06,10.2])
                fake_massive_flag1 = (cat['Z_COMBINE']>0.2) & (cat['Z_COMBINE']<=0.5) & (cat['MASS_MED']>fake_massive_cuts[0]) & (flag_irac==0)
                fake_massive_flag2 = (cat['Z_COMBINE']>0.5) & (cat['Z_COMBINE']<=0.8) & (cat['MASS_MED']>fake_massive_cuts[1]) & (flag_irac==0)
                fake_massive_flag3 = (cat['Z_COMBINE']>0.8) & (cat['Z_COMBINE']<=1.1) & (cat['MASS_MED']>fake_massive_cuts[2]) & (flag_irac==0)
                fake_massive_flag4 = (cat['Z_COMBINE']>1.1) & (cat['Z_COMBINE']<=1.5) & (cat['MASS_MED']>fake_massive_cuts[3]) & (flag_irac==0)
                fake_massive_flag = (fake_massive_flag1 + fake_massive_flag2 + fake_massive_flag3 + fake_massive_flag4).astype(int)
                flag_mag = (cat['i']<23) & (cat['u']<25).astype(int)
                fake_massive_flag = fake_massive_flag*flag_mag
            else:
                fake_massive_flag = np.zeros(len(cat))
            
        #### add fake massive flag
        fake_massive_col = Column(data=fake_massive_flag, name='HIGH_MASS_FLAG')
        if 'HIGH_MASS_FLAG' in cat.keys():
            cat.remove_column('HIGH_MASS_FLAG')
        cat.add_column(fake_massive_col)
        
        ### add morphological flag psf-cmodel mag
        cond_morph = (cat['i_psf']-cat['i_cmodel']<-0.03*cat['i_cmodel']+23.7*0.03).astype(int)
        morph_flag_col = Column(data=cond_morph, name='i_compact_flag')
        if 'i_compact_flag' in cat.keys():
            cat.remove_column('i_compact_flag')
        cat.add_column(morph_flag_col)
        
        # write catalog
        if add_sfq == 'sfq_only':
            cat.write(cat_full_name.replace('.fits','_sfq_only.fits'),overwrite=True)
        else:
            cat.write(cat_full_name.replace('.fits','_sfq_added_z15.fits'),overwrite=True)


  0%|          | 167/1903725 [00:00<19:00, 1669.72it/s]

/Users/lejay/research/lephare_dev/my_code/graham_output_cats_new/pcat_COSMOS_deep_cat_out_nomaglimit_nonir_il_nz_prior_formass.fits
COSMOS_deep nonir free
fc_weighted


100%|██████████| 1903725/1903725 [02:58<00:00, 10664.55it/s]


1903725


  0%|          | 1153/1903725 [00:00<02:45, 11526.41it/s]

...

1492730


  0%|          | 2755/1492730 [00:00<01:49, 13580.57it/s]

onebin_bal


100%|██████████| 1492730/1492730 [02:05<00:00, 11911.02it/s]


1492730


#### show gal count in each field

In [9]:
cat_name_full = '/Users/lejay/research/lephare_dev/my_code/graham_output_cats_new/pcat_XMM-LSS_deep_cat_out_nomaglimit_noirac_il_nz_prior_formass.fits'
cat = Table.read(cat_name_full)

In [21]:
best_boxes_nuvrk = np.load('best_params_4d_ssfr_fcfm_1.0.npy') # overall sfq boundary from 0.1-1.1 references
print(best_boxes_nuvrk)

best_boxes_nuvrk = np.load('best_params_4d_ssfr_fcfm_1.0_z15.npy') # overall sfq boundary from 0.1-1.1 references
print(best_boxes_nuvrk)

[ 3.752e+00  2.358e+00  1.361e+00 -2.440e-01  2.389e-03]
[ 3.678  2.845  1.036 -0.038  0.008]
